In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from cvx.covariance.ewma import iterated_ewma, _ewma_mean, _ewma_cov
from cvx.covariance.combination import from_sigmas
from cvx.covariance.regularization import regularize_covariance, em_regularize_covariance
from experiments.utils.experiment_utils import *
from experiments.utils.factor_model import *

sns.set()
sns.set(font_scale=1.5)

%load_ext autoreload
%autoreload 2

In [44]:
returns = pd.read_csv(
    "../data/49_industries.csv", index_col=0, header=0, parse_dates=True
)

In [40]:
iewma_pair = (63, 125)
iewma = list(
    iterated_ewma(
        returns,
        vola_halflife=iewma_pair[0],
        cov_halflife=iewma_pair[1],
        min_periods_vola=0,
        min_periods_cov=0,
        nan_to_num=True,
    )
)
iewma = {iterate.time: iterate.covariance for iterate in iewma}

covariance = iewma[returns.index[-1]]

In [52]:
iewma_pair = (63, 125)


In [156]:
# 10 percent random nans

np.random.seed(0)
mask = np.random.choice([True, False], size=returns.iloc[:5000].shape, p=[0.1, 0.9])

returns_nans = returns.copy()
returns_nans.iloc[:5000][mask] = np.nan

/var/folders/mr/mp0qg4bx6sj56pg6fs3s65kc0000gr/T/ipykernel_18032/43179705.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  returns_nans.iloc[:5000][mask] = np.nan
/var/folders/mr/mp0qg4bx6sj56pg6fs3s65kc0000gr/T/ipykernel_18032/43179705.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  returns_nans.iloc[:5000][mask] = np.nan


In [53]:
times = returns_nans.index

In [75]:
iewma = list(
    iterated_ewma(
        returns,
        vola_halflife=iewma_pair[0],
        cov_halflife=iewma_pair[1],
        min_periods_vola=0,
        min_periods_cov=0,
        nan_to_num=True,
    )
)
iewma = {iterate.time: iterate.covariance for iterate in iewma}


In [168]:
iewma_nans = list(
    iterated_ewma(
        returns_nans,
        vola_halflife=iewma_pair[0],
        cov_halflife=iewma_pair[1],
        min_periods_vola=0,
        min_periods_cov=0,
        nan_to_num=True,
        mean=True,
    )
)
iewma_nans = {iterate.time: iterate.covariance for iterate in iewma_nans}


/Users/kasper/Documents/Stanford/Research/My papers/w_it_ewma/publicRepo/cov_pred_finance/cvx/covariance/ewma.py:132: RuntimeWarning: divide by zero encountered in divide
  
/Users/kasper/Documents/Stanford/Research/My papers/w_it_ewma/publicRepo/cov_pred_finance/cvx/covariance/ewma.py:133: RuntimeWarning: invalid value encountered in multiply
  # wherever vola is not zero: adj = clip((returns / vola), clip_at=clip_at)


In [162]:
iewma_nans[times[0]].isna().all().sum()

5

In [164]:
returns_nans.iloc[0].isna()

Agric    False
Food     False
Soda     False
Beer     False
Smoke    False
Toys     False
Fun      False
Books    False
Hshld    False
Clths    False
Hlth     False
MedEq    False
Drugs    False
Chems    False
Rubbr     True
Txtls     True
BldMt     True
Cnstr    False
Steel    False
FabPr    False
Mach     False
ElcEq    False
Autos    False
Aero     False
Ships    False
Guns     False
Gold     False
Mines    False
Coal     False
Oil      False
Util     False
Telcm    False
PerSv    False
BusSv    False
Hardw     True
Softw    False
Chips    False
LabEq    False
Paper    False
Boxes    False
Trans    False
Whlsl    False
Rtail    False
Meals     True
Banks    False
Insur    False
RlEst    False
Fin      False
Other    False
Name: 1969-07-01 00:00:00, dtype: bool

In [161]:
returns_nans.isna().sum().sum()

24636

In [155]:
returns_nans.isna().sum().sum() 

0

In [121]:
(iewma_nans[times[6000]]-iewma[times[6000]]).abs().sum().sum()

2.9348986264781627e-05

In [107]:
iewma[times[0]]

Agric      Food          Soda      Beer     Smoke      Toys  \
Agric  0.000029  0.000030 -1.026000e-05  0.000023  0.000045  0.000049   
Food   0.000030  0.000030 -1.045000e-05  0.000024  0.000046  0.000049   
Soda  -0.000010 -0.000010  3.610000e-06 -0.000008 -0.000016 -0.000017   
Beer   0.000023  0.000024 -8.170000e-06  0.000018  0.000036  0.000039   
Smoke  0.000045  0.000046 -1.596000e-05  0.000036  0.000071  0.000076   
Toys   0.000049  0.000049 -1.710000e-05  0.000039  0.000076  0.000081   
Fun    0.000062  0.000063 -2.185000e-05  0.000049  0.000097  0.000103   
Books -0.000002 -0.000002  7.600000e-07 -0.000002 -0.000003 -0.000004   
Hshld  0.000037  0.000038 -1.311000e-05  0.000030  0.000058  0.000062   
Clths  0.000021  0.000021 -7.220000e-06  0.000016  0.000032  0.000034   
Hlth  -0.000049 -0.000049  1.710000e-05 -0.000039 -0.000076 -0.000081   
MedEq  0.000028  0.000028 -9.690000e-06  0.000022  0.000043  0.000046   
Drugs  0.000036  0.000036 -1.254000e-05  0.000028  0.000055  0.000059   
Chems  0.000019  0.000020 -6.840000e-06  0.000015  0.000030  0.000032   
Rubbr  0.000076  0.000078 -2.679000e-05  0.000061  0.000118  0.000127   
Txtls  0.000032  0.000033 -1.140000e-05  0.000026  0.000050  0.000054   
BldMt  0.000043  0.000043 -1.501000e-05  0.000034  0.000066  0.000071   
Cnstr  0.000017  0.000017 -5.890000e-06  0.000013  0.000026  0.000028   
Steel  0.000058  0.000059 -2.052000e-05  0.000046  0.000091  0.000097   
FabPr  0.000067  0.000068 -2.356000e-05  0.000053  0.000104  0.000112   
Mach   0.000045  0.000046 -1.596000e-05  0.000036  0.000071  0.000076   
ElcEq -0.000089 -0.000090  3.116000e-05 -0.000071 -0.000138 -0.000148   
Autos  0.000019  0.000019 -6.650000e-06  0.000015  0.000029  0.000031   
Aero  -0.000004 -0.000004  1.520000e-06 -0.000003 -0.000007 -0.000007   
Ships  0.000028  0.000028 -9.690000e-06  0.000022  0.000043  0.000046   
Guns  -0.000021 -0.000021  7.410000e-06 -0.000017 -0.000033 -0.000035   
Gold   0.000018  0.000018 -6.270000e-06  0.000014  0.000028  0.000030   
Mines  0.000043  0.000044 -1.520000e-05  0.000034  0.000067  0.000072   
Coal   0.000130  0.000133 -4.579000e-05  0.000104  0.000202  0.000217   
Oil    0.000057  0.000058 -1.995000e-05  0.000045  0.000088  0.000094   
Util   0.000011  0.000011 -3.800000e-06  0.000009  0.000017  0.000018   
Telcm -0.000042 -0.000043  1.482000e-05 -0.000034 -0.000066 -0.000070   
PerSv  0.000084  0.000085 -2.945000e-05  0.000067  0.000130  0.000139   
BusSv  0.000122  0.000124 -4.294000e-05  0.000097  0.000190  0.000203   
Hardw  0.000014  0.000014 -4.750000e-06  0.000011  0.000021  0.000022   
Softw  0.000202  0.000206 -7.106000e-05  0.000161  0.000314  0.000337   
Chips  0.000037  0.000038 -1.311000e-05  0.000030  0.000058  0.000062   
LabEq  0.000051  0.000052 -1.805000e-05  0.000041  0.000080  0.000085   
Paper -0.000026 -0.000027  9.310000e-06 -0.000021 -0.000041 -0.000044   
Boxes  0.000016  0.000016 -5.510000e-06  0.000012  0.000024  0.000026   
Trans  0.000044  0.000045 -1.539000e-05  0.000035  0.000068  0.000073   
Whlsl  0.000021  0.000021 -7.410000e-06  0.000017  0.000033  0.000035   
Rtail  0.000024  0.000024 -8.360000e-06  0.000019  0.000037  0.000040   
Meals  0.000091  0.000093 -3.211000e-05  0.000073  0.000142  0.000152   
Banks  0.000012  0.000012 -4.180000e-06  0.000009  0.000018  0.000020   
Insur  0.000016  0.000016 -5.700000e-06  0.000013  0.000025  0.000027   
RlEst  0.000077  0.000079 -2.717000e-05  0.000061  0.000120  0.000129   
Fin    0.000002  0.000002 -5.700000e-07  0.000001  0.000003  0.000003   
Other  0.000199  0.000203 -7.011000e-05  0.000159  0.000310  0.000332   

            Fun         Books     Hshld     Clths  ...         Boxes  \
Agric  0.000062 -2.160000e-06  0.000037  0.000021  ...  1.566000e-05   
Food   0.000063 -2.200000e-06  0.000038  0.000021  ...  1.595000e-05   
Soda  -0.000022  7.600000e-07 -0.000013 -0.000007  ... -5.510000e-06   
Beer   0.000049 -1.720000e-06  0.000030  0.000016  ...  1.247000e-05   